In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import geojson

In [ ]:
# helper function to get  each women's time mandate_end as a column
def get_last_year_in_office(df):
    functions = {'hrllo': (' (assassinated)','',regex=False), 
                    'test': ('Incumbent',np.NaNm),
                     fillna: df['Mandate start']}


In [22]:
# read in female heads of state dataset
df = pd.read_csv('data/female_heads_of_state.csv')
countries = pd.read_csv('data/list_of_countries.csv')

In [24]:
# add in the Alpha-3 code
df = df.merge(countries[['Country','Alpha-3 code']],on='Country')

# add colunms that have the start and end years of their terms
df['start_year'] = df['Mandate start'].apply(lambda x: f'19{x[-2:]}' if int(x[-2:]) > 39 else f'20{x[-2:]}')
df['Mandate end'] = df['Mandate end'].str.replace(' (assassinated)','',regex=False)
df['Mandate end'] = df['Mandate end'].replace('Incumbent',np.NaN)
df['Mandate end'] = df['Mandate end'].fillna(df['Mandate start'])
df['end_year'] = df['Mandate end'].apply(lambda x: f'19{x[-2:]}' if int(x[-2:]) > 39 else f'20{x[-2:]}')

In [25]:
# make it so that each year of each leader's term is a row
df['Year'] = [pd.date_range(x, y,freq='YS',inclusive='both') for x, y in zip(df.start_year, df.end_year)]
df = df.explode('Year')
df['Year'] = df['Year'].astype(str).apply(lambda x:x[:4])

In [27]:
# read in geojson for choropleth 
with open('data/countries.geojson') as f:
    gj = geojson.load(f)

# add in the 'id' which allows plotly to associate the cordinates to a teh countries in my dataframe
for feature in gj['features']:
    feature['id'] = feature['properties']['ISO_A3']

In [ ]:
# only do three years to save memory b/c choropleth is to large otherwise
df_less = df[['Name','Country','Office','Head of state or government','Alpha-3 code','Year']].sort_values('Year')
df_less = df_less[df_less['Year'].isin([str(year) for year in range(2020,2023)])]

### Choropleth visualzation

In [ ]:
# without animation (only 2022)
fig = px.choropleth_mapbox(df[[df_less]==2022], geojson=gj, locations='Alpha-3 code', 
                    color='Head of state or government', mapbox_style='carto-positron', zoom=1,
                           color_discrete_sequence=px.colors.qualitative.Prism,
                          hover_data=['Name','Country','Office'])

fig.show()

In [45]:
# with animation (2020 - 2023)
fig = px.choropleth_mapbox(df_less, geojson=gj, locations='Alpha-3 code', 
                    color='Head of state or government', mapbox_style='carto-positron', zoom=1,
                           color_discrete_sequence=px.colors.qualitative.Prism,
                          hover_data=['Name','Country','Office'],animation_frame='Year')

fig.show()

### Correlation between percent women in congress and democracy index

In [5]:
# read in democracy index data
democracy_indx = pd.read_csv('democracy_index_data.csv')
# read in the percent women in parliments data
women_in_parliments =  pd.read_csv('data/world_data.csv')


In [8]:
women_in_parliments

,COUNTRY,LOWER_OR_SINGLE_HOUSE_ELECTIONS,LOWER_OR_SINGLE_HOUSE_SEATS,LOWER_OR_SINGLE_HOUSE_WOMEN,LOWER_OR_SINGLE_HOUSE_PERCENT_W,UPPER_HOUSE_OR_SENATE_ELECTIONS,UPPER_HOUSE_OR_SENATE_SEATS,UPPER_HOUSE_OR_SENATE_WOMEN,UPPER_HOUSE_OR_SENATE_PERCENT_W
0,Sweden,1998-09-01,349.0,149.0,427.0,1970-01-01,0.0,0.0,0.0
1,Denmark,1998-03-01,179.0,67.0,374.0,1970-01-01,0.0,0.0,0.0
2,Finland,1999-03-01,200.0,73.0,365.0,1970-01-01,0.0,0.0,0.0
3,Netherlands,1998-05-01,150.0,54.0,360.0,1999-05-01,75.0,20.0,267.0
4,Norway,2001-09-01,165.0,59.0,358.0,1970-01-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
53489,Haiti,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0
53490,Venezuela Bolivarian Republic of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53491,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53492,Myanmar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
women_in_parliments['LOWER_OR_SINGLE_HOUSE_ELECTIONS'].astype(str).apply(lambda x:x[:4])


0        1998
1        1998
2        1999
3        1998
4        2001
         ... 
53489     nan
53490     nan
53491     nan
53492     nan
53493     nan
Name: LOWER_OR_SINGLE_HOUSE_ELECTIONS, Length: 53494, dtype: object

In [10]:
# add a year column
women_in_parliments['LOWER_OR_SINGLE_HOUSE_ELECTIONS_YEAR'] = women_in_parliments['LOWER_OR_SINGLE_HOUSE_ELECTIONS'].astype(str).apply(lambda x:x[:4])
women_in_parliments['UPPER_HOUSE_OR_SENATE_ELECTIONS_YEAR'] = women_in_parliments['UPPER_HOUSE_OR_SENATE_ELECTIONS'].astype(str).apply(lambda x:x[:4])


In [11]:
years_list = [str(year) for year in range(2010,2023)]
years_list.extend(['200'])

In [3]:
# get list of years
years_list = [for year in range(2010,2023)]
# melt dataframe from wide to long
democracy_indx.melt(id_vars=['Region','2022 rank','Country','Regime type','five_year_mean'],value_vars=[for year in range(2010,2023)])

,Region,2022 rank,Country,Regime type,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2008,2006,five_year_mean
0,North America,12,Canada,Full democracy,8.88,8.87,9.24,9.22,9.15,9.15,9.15,9.08,9.08,9.08,9.08,9.08,9.08,9.07,9.07,9.126
1,North America,30,United States,Flawed democracy,7.85,7.85,7.92,7.96,7.96,7.98,7.98,8.05,8.11,8.11,8.11,8.11,8.18,8.22,8.22,7.934
2,Western Europe,20,Austria,Full democracy,8.20,8.07,8.16,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,8.246
3,Western Europe,36,Belgium,Flawed democracy,7.64,7.51,7.51,7.64,7.78,7.78,7.77,7.93,7.93,8.05,8.05,8.05,8.05,8.16,8.15,7.644
4,Western Europe,37,Cyprus,Flawed democracy,7.38,7.43,7.56,7.59,7.59,7.59,7.65,7.53,7.40,7.29,7.29,7.29,7.29,7.70,7.60,7.552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Sub-Saharan Africa,92,Tanzania,Hybrid regime,5.10,5.10,5.10,5.16,5.41,5.47,5.76,5.58,5.77,5.77,5.88,5.64,5.64,5.28,5.18,5.248
163,Sub-Saharan Africa,130,Togo,Authoritarian,2.99,2.80,2.80,3.30,3.10,3.05,3.32,3.41,3.45,3.45,3.45,3.45,3.45,2.43,1.75,3.010
164,Sub-Saharan Africa,99,Uganda,Hybrid regime,4.55,4.48,4.94,5.02,5.20,5.09,5.26,5.22,5.22,5.22,5.16,5.13,5.05,5.03,5.14,4.946
165,Sub-Saharan Africa,78,Zambia,Hybrid regime,5.80,5.72,4.86,5.09,5.61,5.68,5.99,6.28,6.39,6.26,6.26,6.19,5.68,5.25,5.25,5.392
